# Chord Progression Frequency Analysis
This python notebook will look into the frequency of chord progressions in a composer's repertoire.



## Table of Contents:

* [Preprocessing](#preprocessing)
    * [Sequence Dictionary](#seq-dict)
    * [Loading Datasets](#loading-datasets)
* [Method 1 - Counting Overall Percentage](#method-1)
    * [Problems](#method-1-problems)
* [Method 2 - Provide percentages at every chord](#method-2)
    * [Problems](#method-2-problems)
* [Method 3 - Kernel Frequency Analysis](#method-3)
    * [Problems](#method-3-problems)
* [Example Likelihood Analysis](#example)

# Preprocessing  <a class="anchor" name="preprocessing"></a>

Before we begin the analysis, we will create a couple of helper functions to conduct the rest of the frequency analysis methods.

## Sequence Dictionary <a class="anchor" name="seq-dict"></a>
The first step is to determine the frequency of an n-long sequence in a text file formatted as in the List Generator notebook.

In [1]:
def build_n_sequence_dictionary(n : int, txt : str) -> dict:
    """
    Returns a dict counting appearances of n-long progressions
    
    Parameters
    ----------
    n : int
        The length of progressions to count
    txt : str
        A comma-separated string of roman numerals
    
    Returns
    -------
    dict 
        Maps sequences in txt of length n to the number of times
        they appear in txt
    """
    seq_dict = dict()
    
    pieces = txt.split('\n')
    # For every song in the string, which are separated by '\n'
    for piece in pieces:
        roman_numerals = piece.split(',')
        # For every sequence of n roman numerals
        for i in range(len(roman_numerals) - n):
            seq = tuple(roman_numerals[i:i+n])
            # Increment the count for this specific sequence
            seq_dict[seq] = seq_dict.get(seq, 0) + 1
    return seq_dict            

In [2]:
# test build_n_sequence_dictionary()

# The following text is the beginning of 'monteverdi/madrigal.3.1.rntxt'
test_txt = "vi,V,I,IV,I,V,V,i,V,i,VI,V,i,V,i,i,i,I,IV,ii,vii,I,vi,I,I,IV,vii"
seq_dict = build_n_sequence_dictionary(2, test_txt)

# Show the most frequently found chord progressions of length 2
print(sorted(seq_dict.items(), key=lambda kv:(-kv[1], kv[0]))[:5])

## Loading the Datasets  <a name="loading-datasets"></a>

The next helper function will load the datasets that are generated from the `List Generator` Jupyter notebook. Given the desired composer and whether or not we want to use the dataset that encodes inversions, this method will return the contents of the respective file. 

In [3]:
def get_dataset(composer: str, with_inversions=False) -> str:
    '''
    Returns the content of the txt file for the composer's repertoire
    
    Parameters
    ----------
    composer : str
        The composer which has been encoded
    with_inversion : bool, optional
        Whether to fetch the dataset with or without inversions
        (default is False for no inversions)
    
    Returns
    -------
    str
        A comma-separated string encoding the composer's repertoire
    '''
    
    file_location = "datasets/{}-dataset-{}.txt".format(
        "inv" if with_inversions else "simple", 
        composer)
    try:
        return open(file_location, "r").read()
    except FileNotFoundError:
        raise NotImplementedError("The composer you've entered is not in the database.")

# Method 1 - Counting and getting an overall percentage <a class="anchor" name="method-1"></a>

The first method we implemented would look at the entire chord progression the user would provide and count the number of occurrences of the full progression occurs in the specified composer's repertoire.


To interpret this as a preference, we count the total number of `n`-long chords in the composer's repertoire, where `n` is the number of chords in the user's chord progression. Then, we count the number of times that the user's provided chord progression occurs in the composer's repertoire. Finally, we divide the latter by the former to get a fraction we interpret as the preference of the user's full chord progression to others of the same length. The image below illustrates this calculation.

<img src="images/method-1.svg" width="400px"></img>

In [4]:
def progression_count_and_percent(prog: str, composer: str, with_inversions=False, verbose=False) -> (int, int):
    '''
    Returns a tuple with the number of times the given progression is used, and 
    the percentage used over other progressions of the same length
    
    Parameters
    ----------
    prog : str
        a chord progression formatted as roman numerals separated by commas
    composer : str
        the last name of the composer to reference
    with_inversions : bool, optional
        False for no inversions, True for inversions (default False)
    verbose : bool, optional
        True to print the information in readable format (default False)
        
    Returns
    -------
    int
        number of occurances of the given progression
    int 
        fraction of times this progression was used over others of the same length
    '''
    # removes any whitespace
    prog = prog.replace(' ', '')

    txt = get_dataset(composer, with_inversions)
        
    # creates the dictionary given the progression
    prog_list = tuple(prog.split(','))
    seq_dict = build_n_sequence_dictionary(len(prog_list), txt)
    total_progs = sum(seq_dict.values())
    num_times = seq_dict.get(prog_list, 0)
    print(total_progs)
    # Prints the information in a readable format before returning the values
    if verbose:
        if num_times == 0:
            print('{} never uses [{}] in their repertoire.'.format(composer, prog))
        elif num_times == 1:
            print('{} uses [{}] {} time in their repertoire.'.format(composer, prog, num_times))
        else: 
            print('{} uses [{}] {} times in their repertoire.'.format(composer, prog, num_times))
        print('Compared to other length-{} progressions, [{}] is used {:.2f}% of the time.'.format(len(prog_list), prog, num_times/total_progs*100))
        
    return num_times, num_times/total_progs

In [5]:
# test without inversion
print("Test Without Inversions")
prog = 'V,I'
print(progression_count_and_percent(prog, 'monteverdi', verbose=True))

# test with inversion
print("\nTest With Inversions")
prog = 'V(0),I(0)'
print(progression_count_and_percent(prog, 'monteverdi', with_inversions=True, verbose=True))

Test Without Inversions
6613
monteverdi uses [V,I] 446 times in their repertoire.
Compared to other length-2 progressions, [V,I] is used 6.74% of the time.
(446, 0.06744291546952971)

Test With Inversions
6613
monteverdi uses [V(0),I(0)] 312 times in their repertoire.
Compared to other length-2 progressions, [V(0),I(0)] is used 4.72% of the time.
(312, 0.04717979736881899)


### Problems with Method 1 <a name="method-1-problems"></a>

First, this approach cannot tell us which part of the chord progression was the least like work made by the composer. At worst, it simply tells the user whether it even appears in the composer's repertoire at all.

Second, the choice to divide the number of occurrences by the number of chords of the same length is arbitrary. For a large enough data-set, this would simply be dividing the number of occurrences by the number of chords in the composers repertoire, and thus the score does not is not of much substance.

Finally, the key issue with the first option is that if the length of the user's repertoire is too long, or if it doesn't appear in the composer's repertoire, then it is considered impossible.

# Method 2 - Provide Frequency with Full Context <a class="anchor" name="method-2"></a>

To address the first issue with Method 1 where the analysis method does not tell the user which part of the chord progression contributes to being more or less similar to the given composer, we will use a method that interprets frequencies we gather when counting as probabilities. We will be returning a list of numbers, which tells us how often the composer follows a given sequence of chords by the next chord. In otherwords, what is the probability of the first `i` chords being followed by the `ith` chord.

Below is an example illustrating the probability that the progression `[V,I]` is followed by the chord `[IV]`.

<img src="images/method-2-calculation.svg" width="400px"/>

Additionally, this method fixes starting position of the context to the beginning of the user's provided chord progression. Below illustrates this process and the list that is returned by method 2.

<img src="images/method-2-process.svg" width="400px"/>

In [6]:
def progression_probability(prog: str, composer: str, with_inversions=False, verbose=False) -> float:
    '''
    Returns a list of frequencies for substrings of the progression fixed to the beginning

        Given a chord progression as a string of roman numerals separated by 
    commas (followed by perentheses containing inversion number if the parameter 
    with_inversions=True), and a composer's last name, returns a list containing
    the probability that the i-th chrd in the progression follows the i-1 chords
    that precede it using it's frequency in the given composer's repertoire.
        If verbose is true, prints the information in readable format to the 
    console in addition to returning the list of frequencies
    
    Parameters
    ----------
    prog : str
        a chord progression formatted as roman numerals separated by commas
    composer : str
        the last name of the composer to reference
    with_inversions : bool, optional
        False for no inversions, True for inversions (default False)
    verbose : bool, optional
        True to print the information in readable format (default False)

    Returns
    -------
    float[]
        A list of frequencies of the i-th chrd in the progression is followed by
        the i-1 chords in the composer's encoded repertoire
    '''
    # removes any whitespace
    prog = prog.replace(' ', '')
    
    txt = get_dataset(composer, with_inversions)    
    
    prog_prob = [] # empty list to contain tuples of chord progression and probabilities
    prog_list = tuple(prog.split(','))
    # creates the dictionary given the progression
    seq_dict = build_n_sequence_dictionary(len(prog_list), txt) 
    
    # values to be modified as i-1 chords in progression are given
    total_progs = sum(seq_dict.values())
    chords_to_check = seq_dict.keys()
    
    # flag used when progression is not used in repertoire
    shortcut = False
    
    for i in range(len(prog_list)):
        if shortcut:
            prog_prob.append((prog_list[:i+1], 0.))
            continue
            
        same_start_chords = list(filter(lambda a: (a[:i+1] == prog_list[:i+1]), chords_to_check))
        
        # count number of same_start_chords in repertoire
        progs = 0
        for chord in same_start_chords:
            progs += seq_dict[chord]
        if progs == 0:
            # shows actual chords that follow
            shortcut = True
        
        # add probability to list
        prog_prob.append((prog_list[:i+1], progs/total_progs))
        # update values assuming i-1 chords are given
        total_progs = progs
        chords_to_check = same_start_chords
    
    if verbose:
        # msg for first chord in progression
        print('{} starts a progression with [{}] {:.2f}% of the time'.format(
            composer, prog_list[0], prog_prob[0][1]*100))
        # msg for the rest of the chords
        for i in range(1, len(prog_list)):
            # formats the preceding chords as a comma separated string
            preceding_chords = ','.join([chord for chord in prog_list[:i]])
            print('{} follows [{}] with [{}] {:.2f}% of the time'.format(
                composer, preceding_chords, prog_list[i], prog_prob[i][1]*100))
    return prog_prob

In [7]:
# test without inversion
prog = 'V,I,IV,V,vi,iii,V'
print(progression_probability(prog, 'monteverdi', verbose=True))

monteverdi starts a progression with [V] 22.73% of the time
monteverdi follows [V] with [I] 30.12% of the time
monteverdi follows [V,I] with [IV] 14.19% of the time
monteverdi follows [V,I,IV] with [V] 29.03% of the time
monteverdi follows [V,I,IV,V] with [vi] 11.11% of the time
monteverdi follows [V,I,IV,V,vi] with [iii] 50.00% of the time
monteverdi follows [V,I,IV,V,vi,iii] with [V] 0.00% of the time
[(('V',), 0.2273225755914147), (('V', 'I'), 0.30117160578911095), (('V', 'I', 'IV'), 0.14187643020594964), (('V', 'I', 'IV', 'V'), 0.2903225806451613), (('V', 'I', 'IV', 'V', 'vi'), 0.1111111111111111), (('V', 'I', 'IV', 'V', 'vi', 'iii'), 0.5), (('V', 'I', 'IV', 'V', 'vi', 'iii', 'V'), 0.0)]


In [8]:
# test with inversion
prog = 'V(0),I(0)'
print(progression_probability(prog, 'monteverdi', with_inversions=True, verbose=True))

monteverdi starts a progression with [V(0)] 19.96% of the time
monteverdi follows [V(0)] with [I(0)] 23.64% of the time
[(('V(0)',), 0.1996068350219265), (('V(0)', 'I(0)'), 0.23636363636363636)]


### Problems with Method 2 <a name="method-2-problems"></a>

The largest problem with this method is that for a long enough chord progression would always approach a score of 0%. Because the end result that we wanted was an algorithm that would score how well a given chord progression resembles the work of a composer, this wasn't going to fly. 

We needed an approach that wouldn't look back as far as this algorithm would, which leads us to the third and final method for our project.

#  Method 3 - Sliding Kernel <a class="anchor" name="method-3"></a>

The final method that we implemented was a sliding kernel. Rather than fixing the context's start position and varying the length of the context being used, we will fix the length of the context and vary the starting position of the calculation. This approach resembles a 1-dimensional kernel being applied to the user's chord progression. Below is an illustration that shows this process.

<img src="images/method-3-kernel.svg" width="400px"/>

This sliding kernel is also a step towards addressing the problem of translational invariance. The likelihood that the composer follows VI with a IV does not largely vary based on all of the chords which precede.

Finally, to draw conclusions using the data from this method, we find the minimum output of the kernel. If the this value is below some arbirtrary threshold, we highlight this portion of the progression and suggest that the user reconsider this portion of their chord progression to increase similarity with the composer.

<img src="images/method-3-conclusions.svg" width="400px"/>

In the final application, we set the arbitrary threshold to `5%` or `0.05`. Thus, the final application would rate that this progression as somewhat similar to Monteverdi's repertoire.

In [9]:

def progression_kernel(prog: str, composer: str, with_inversions=False, look_back=2, verbose=False):
    '''
    Returns a list of frequencies of subsets of the progression of fixed length

        Given a progression as a comma-separated string of chords (followed by
    parentheses with the inversion number if the parameter with_inversions=True),
    a composer's last name, and the number of chords to serve as the context of
    the kernel, we apply the sliding kernel to determine the likelihood that the
    composer follows a sequence of chords with the same chord in the given
    progression.
        If verbose is true, prints the information in readable format to the
    console in addition to returning the list of frequencies.

    Parameters
    ----------
    prog : str
        a chord progression formatted as roman numerals separated by commas
    composer : str
        the last name of the composer to reference
    with_inversions : bool, optional
        False for no inversions, True for inversions (default False)
    look_back : int, optional
        (At least 1) The number of chords to serve as the context (default 2)
    verbose : bool, optional
        True to print the information in readable format (default False)

    Returns
    -------
    float[]
        A list of the frequencies for each application of the kernel
    '''
    percentages = []

    # removes any whitespace
    prog = prog.replace(' ', '')

    txt = get_dataset(composer, with_inversions)

    # creates the dictionary given the progression
    prog_list = tuple(prog.split(','))

    if verbose:
        print("Analyzing progression: [{}]".format(prog))

    look_back_dict = build_n_sequence_dictionary(look_back, txt)

    seq_dict = build_n_sequence_dictionary(look_back + 1, txt)

    # For every look_back + 1 sequence in the progression
    for i in range(len(prog_list)-look_back):

        # Total Occurances of look_back
        look_back_occurances = look_back_dict.get((prog_list[i:i+look_back]),0)
        # Num Occurances look back is followed by the next chord
        look_back_with_next_chord = seq_dict.get(prog_list[i:i+look_back+1], 0)

        # Calculate the frequency that a chord follows the n previous
        if look_back_occurances == 0:
            prog_percent = 0
        else:
            prog_percent = look_back_with_next_chord / look_back_occurances

        percentages.append(prog_percent)

        if verbose:
            print("{} follows [{}] with [{}] {:.2f}% of the time".format(
                composer,
                ",".join(prog_list[i:i+look_back]),
                prog_list[i+look_back],
                prog_percent * 100))

    return percentages

In [10]:
print("--  Example 1")
print(progression_kernel('V,I,IV,V,vi,iii,V', "monteverdi",look_back=2,verbose=True))

print("\n--  Example 2")
print(progression_kernel('V,I,IV,V,vi,iii,V'[::-1], "monteverdi",look_back=2,verbose=True))

--  Example 1
Analyzing progression: [V,I,IV,V,vi,iii,V]
monteverdi follows [V,I] with [IV] 14.13% of the time
monteverdi follows [I,IV] with [V] 20.83% of the time
monteverdi follows [IV,V] with [vi] 7.34% of the time
monteverdi follows [V,vi] with [iii] 8.70% of the time
monteverdi follows [vi,iii] with [V] 9.09% of the time
[0.1412556053811659, 0.20833333333333334, 0.07339449541284404, 0.08695652173913043, 0.09090909090909091]

--  Example 2
Analyzing progression: [V,iii,iv,V,VI,I,V]
monteverdi follows [V,iii] with [iv] 0.00% of the time
monteverdi follows [iii,iv] with [V] 100.00% of the time
monteverdi follows [iv,V] with [VI] 0.00% of the time
monteverdi follows [V,VI] with [I] 0.00% of the time
monteverdi follows [VI,I] with [V] 28.57% of the time
[0.0, 1.0, 0.0, 0.0, 0.2857142857142857]


### Problems with Method 3 <a name="method-3-problems"></a>

The main problem with this method is the assumption that frequencies in the repertoire will closely reflect the probability that the given composer will follow some phrase with the next chord.

Additionally, it would be worth taking into account the location of the chord in the overall song. This way, we can account for the timing of chord progressions.

# Example Likelihood Analysis <a name="example"></a>

The basics of this analysis is to look at the list of frequencies that are given by the kernel method above. If any part of the progression is below some arbitrary threshold, `progression_likelihood` says that the progression is unlikely. If all parts of the progression is above a different threshold, it says the progression is fairly likely. Otherwise, it says that it cannot give a strong opinion one way or the other. 

In [11]:
def progression_likelihood (prog: str, composer: str, with_inversions=False, look_back=2):
    
    # Arbitrary Thresholds
    unlikely_threshold = 0.1
    likely_threshold = 0.4
    # Saves the list of frequencies given by the kernel method above
    frequencies = progression_kernel(prog, composer, with_inversions=with_inversions,look_back=look_back, verbose=True)
    
    print("\nThe following are places in your progression which are particularly unlikely:")

    split_progression = prog.split(",")

    for i in range(len(frequencies)):
        frequency = frequencies[i]

        if frequency < 0.05:
            bad_prog = ",".join(split_progression[i:i+look_back+1])
            print ("  Consider looking at the progression: [{}]".format(bad_prog))
            print ("    {} doesn't follow [{}] with [{}] very often."
                .format(composer, ",".join(split_progression[i:i+look_back]), split_progression[i+look_back]))

In [12]:
prog = input("Progression?")

print("")

progression_likelihood(prog, "monteverdi", with_inversions=False,look_back=2)

Progression?V,I,IV,V,vi,iii,V

Analyzing progression: [V,I,IV,V,vi,iii,V]
monteverdi follows [V,I] with [IV] 14.13% of the time
monteverdi follows [I,IV] with [V] 20.83% of the time
monteverdi follows [IV,V] with [vi] 7.34% of the time
monteverdi follows [V,vi] with [iii] 8.70% of the time
monteverdi follows [vi,iii] with [V] 9.09% of the time

The following are places in your progression which are particularly unlikely:


In [13]:
#eof